# copy of generate_error.py from proPTV

In [141]:
# import matplotlib.pyplot as plt
import numpy as np 
from pathlib import Path


from optv.parameters import ControlParams, VolumeParams
from optv.calibration import Calibration
# from optv.orientation import external_calibration, full_calibration

from optv.imgcoord import image_coordinates
from optv.transforms import convert_arr_metric_to_pixel, convert_arr_pixel_to_metric, distorted_to_flat
from optv.orientation import point_positions
from optv.correspondences import MatchedCoords
from optv.tracking_framebuf import TargetArray

# import plotly.express as px
import plotly.figure_factory as ff
# import plotly.graph_objects as go
from scipy.optimize import minimize

import pandas as pd
import plotly.express as px

In [142]:

# copy of SaveLoad.py
# import numpy as np

def LoadMarkerList(cam):
    return np.loadtxt('markers_c'+str(cam)+'.txt')

def SaveMarkerList(data):
    # Format: x0, y0, x1, y1, x2, y2, x3, y3, X, Y, Z, dX, dY, dZ
    return np.savetxt('markers_error.txt',data)



In [143]:

class Parameter:
    cams = [0,1,2,3]
    Vmin = [0,0,0]
    Vmax = [300,300,300]
    N1, N2 = 361, 5

# load parameter
params = Parameter()



In [144]:
cases_path = (Path.cwd().parent / 'cases')
cases_path.exists()

True

In [145]:
cases = list(cases_path.rglob('case_*'))
for c in cases:
    list_files = list(c.rglob('markers*'))
    list_files.sort()
    print(list_files)


[PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c0.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c1.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c2.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c3.txt')]
[PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c0.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c1.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c2.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c3.txt')]
[PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_extrapolation/markers_c0

In [146]:
cases[1].name.split('case_')[-1]

'allmarkers'

In [147]:
def array_to_calibration(x:np.ndarray, cal:Calibration) -> None:
    cal.set_pos(x[:3])
    cal.set_angles(x[3:6])
    cal.set_primary_point(x[6:9])
    cal.set_radial_distortion(x[9:12])
    cal.set_decentering(x[12:14])
    cal.set_affine_trans(x[14:])
    return None

def calibration_to_array(cal:Calibration) -> np.ndarray:
    return np.concatenate([
        cal.get_pos(),
        cal.get_angles(),
        cal.get_primary_point(),
        cal.get_radial_distortion(),
        cal.get_decentering(),
        cal.get_affine(),
    ])

def error_function(x, cal, XYZ, xy, cpar):
    
    array_to_calibration(x, cal)

    targets = convert_arr_metric_to_pixel(
        image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
    cpar,
    )
    # err = np.sum(np.abs(xy - targets))
    err = np.sum((xy - targets)**2)
    # print(err)
    return err

def targetize(detects, approx_size, sumg=10):
    """
    Creates a correct TargetArray object with the detected positions and some
    placeholder values for target parameters that I don't use.
    
    Arguments:
    detects - (n,2) array, pixel coordinates of a detected target.
    approx_size - a value to use for the pixel size placeholders.
    sumg - a value to use for the sum of grey values placeholder.
        Default: 10.
    """
    targs = TargetArray(len(detects))
    
    tnum = 0
    for t, pos in zip(targs, detects):
        t.set_pos(pos)
        t.set_pnr(tnum)
        t.set_sum_grey_value(sumg) # whatever
        t.set_pixel_counts(approx_size**2 * 4, approx_size*2, approx_size*2)
        t.set_tnr(-1) # The official "correspondence not found" that 
                               # the rest of the code expects.
        tnum += 1
    
    return targs

def pixel_to_3d(markers, cpar, cals, vpar):
    """ converts numpy array of size (2,) from pixel to flat coordinates"""
    detected = []
    corrected = []
    pnrs = []
    for cix in range(cpar.get_num_cams()):
        targs = targetize(markers[cix][:,:2], 1,1)
        # targs.sort_y()  # not sure why it matters but it does
        
        detected.append(targs)
        pnrs.append([t.pnr() for t in targs])

        # mc = 
        # _, pnr = mc.as_arrays()
        # pnrs.append(pnr)
        corrected.append(MatchedCoords(targs, cpar, cals[cix]))

    flat = np.array([corrected[cix].get_by_pnrs(np.array(pnrs[cix])) \
            for cix in range(len(cals))])

    pos3d, rcm = point_positions(flat.transpose(1,0,2), cpar, cals, vpar)

    return pos3d, rcm

In [150]:
n_cams = len(params.cams)

cpar = ControlParams(n_cams)
cpar.read_control_par(b"parameters/ptv.par")

vpar = VolumeParams()
vpar.read_volume_par(b"parameters/criteria.par")

# Calibration initial guess 

cals = []
for i_cam in range(n_cams):
    cal = Calibration()
    tmp = cpar.get_cal_img_base_name(i_cam)
    cal.from_file(tmp + b".ori", tmp + b".addpar")
    print(cal.get_pos(), cal.get_angles())
    cals.append(cal)



# prepare all markers for the rms error 
list_files = list(cases[1].rglob('markers*'))
all_markers = [np.loadtxt(_) for _ in list_files]
all_XYZ = all_markers[0][:,2:]
all_ID = np.argwhere((all_XYZ[:,0]>-1))[:,0]

for case in cases:
    # if 'allmarkers' not in case.name:
    #     continue

    list_files = list(case.rglob('markers*'))
    list_files.sort()
    case_name = case.name.split('case_')[-1] # 'interpolation', ...
    print(case_name)
    # print([_.name for _ in list_files])
    
    # load marker
    markers = [np.loadtxt(_) for _ in list_files]
    
    for c in params.cams:
        # print(f" Camera {c}\n")

        XYZ = markers[c][:,2:]
        xy = markers[c][:,:2]
        ID = np.argwhere((XYZ[:,0]>-1))[:,0]

        cal = cals[c]
        # print what you get to see it's still a valid guess
        cal.get_pos(), cal.get_angles()

        
        # We could use this step only if we do not have a good
        # initial guess, but we have one from the previous step

        
        four_points = xy[[0,int(ID.max()/4),int(ID.max()*3/4),ID.max()],:] # choose manually
        ref_pts = XYZ[[0,int(ID.max()/4),int(ID.max()*3/4),ID.max()],:]


        targets = convert_arr_metric_to_pixel(
            image_coordinates(ref_pts, cal, cpar.get_multimedia_params()),
        cpar,
        )
        print(f"Before: {four_points - targets}")


        # external_calibration(cal, ref_pts, four_points, cpar)


        x0 = calibration_to_array(cal)
        # print(x0)
        sol = minimize(error_function, x0, args=(cal, XYZ, xy, cpar), method='Nelder-Mead', tol=1e-11)

        array_to_calibration(sol.x, cal)

        targets = convert_arr_metric_to_pixel(
            image_coordinates(ref_pts, cal, cpar.get_multimedia_params()),
        cpar,
        )
        print(f"After: {four_points - targets}")

        targets = convert_arr_metric_to_pixel(
            image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
        cpar,
        )

        # px.scatter(x=xy[:,0], y=xy[:,1], color=ID).show()
        # fig = ff.create_quiver(x=xy[:,0], y=xy[:,1], u=targets[:,0]-xy[:,0], v=targets[:,1]-xy[:,1], scale=5)
        # fig.show()

        # Not sure I understand it correctly, we calibrate with 
        # some markers but always compare with the full set


    # Note that we always use allmarkers for comparison:
    # newXYZ, rcm = pixel_to_3d(all_markers, cpar, cals, vpar)

    newXYZ, rcm = pixel_to_3d(markers, cpar, cals, vpar)
    # errors = newXYZ - all_XYZ
    errors = newXYZ - XYZ

    print(f" Error rms: {np.sqrt(np.sum(errors**2))}")

    # print(rcm)

    newxyz = pd.DataFrame(all_XYZ, columns=['x','y','z'])
    newxyz['id'] = all_ID
    px.scatter_3d(x=newxyz['x'], y=newxyz['y'], z=newxyz['z'], color=newxyz['id']).show()

    
    newxyz = pd.DataFrame(newXYZ, columns=['x','y','z'])
    newxyz['id'] = range(len(newXYZ))
    px.scatter_3d(x=newxyz['x'], y=newxyz['y'], z=newxyz['z'], color=newxyz['id']).show()

    # get 3D positions from plate
    # # triangulate 3D position
    # camPs = [ np.asarray([markers[0][i,:2],markers[1][i,:2],markers[2][i,:2],markers[3][i,:2]]) for i in range(len(markers[0]))]
    # P = np.asarray([NewtonSoloff_Triangulation(setP, ax, ay, params)[0] for setP in camPs])

    # # compare error
    # error = XYZ-P
    # output = np.zeros([361*5,8+3+3])
    # for i in range(len(params.cams)):
    #     output[:,2*i:2*(i+1)] = np.loadtxt('markers_c'+str(params.cams[i])+'.txt')[:,:2]
    # output[:,8:11] = XYZ
    # output[:,11:] = error
    # # SaveMarkerList(output)

    # err = np.linalg.norm(error,axis=1)
    # mean_err = np.mean(err)
    # std_err = np.std(err)
    # print('MAE: ', mean_err, ' +- ' , std_err )
    # import matplotlib.pyplot as plt

    # axis = plt.figure().add_subplot(111, projection='3d')
    # axis.scatter(P[:,2], P[:,0], P[:,1],c='red',s=5)
    # axis.scatter(XYZ[ID,2], XYZ[ID,0], XYZ[ID,1],c='black',s=5)
    # axis.set_xlabel('Z',fontsize=13), axis.set_ylabel('X',fontsize=13), axis.set_zlabel('Y',fontsize=13)
    # plt.tight_layout()
    # plt.show()

    # axis = plt.figure().add_subplot(111, projection='3d')
    # axis.quiver(P[:,2],P[:,0],P[:,1], P[:,2]-XYZ[ID,2], P[:,0]-XYZ[ID,0], P[:,1]-XYZ[ID,1],color='red',length=10,normalize=True)
    # axis.set_xlabel('Z',fontsize=13), axis.set_ylabel('X',fontsize=13), axis.set_zlabel('Y',fontsize=13)
    # plt.tight_layout()
    # plt.show()

[ 322.38251339    4.4040676  1031.76370716] [ 0.1884708   0.20904414 -0.03438548]
[-1.72196260e-01  2.96220478e+02  1.03049543e+03] [-0.2028321  -0.18304368 -0.02799897]
[ 323.84409902  298.102586   1028.23341819] [-0.19496146  0.21075588  0.0246242 ]
[-1.32199240e-01  1.49476157e+01  1.02278223e+03] [ 0.17005024 -0.1784537   0.00354099]
interpolation
Before: [[-3.19232797 -0.09974858]
 [ 0.0096724   3.82059895]
 [ 0.56604697 -3.24775243]
 [ 5.05276383  0.65211193]]
After: [[-1.53650842  3.12003714]
 [-0.96950405  0.27950687]
 [-0.24685225  0.79716483]
 [ 1.339902    2.23122291]]
Before: [[ 4.98487101 -8.00833928]
 [-0.41474971  0.99048098]
 [ 1.58112439  2.99648186]
 [ 7.06044941  1.57179241]]
After: [[ 1.87451371 -0.17870081]
 [ 0.84242104  0.188157  ]
 [-0.31037356  0.61416149]
 [ 2.66102891  2.86960713]]
Before: [[ 1.90844903 -6.76866576]
 [ 1.35766831  5.44138695]
 [ 2.08228527  3.6992673 ]
 [ 9.0635969   1.22556345]]
After: [[ 1.28433413  0.32900924]
 [-0.74419649  0.72186212]
 [

allmarkers
Before: [[-1.53650842  3.12003714]
 [ 0.57513723 -0.24719721]
 [ 0.45559652  0.26383089]
 [ 1.339902    2.23122291]]
After: [[ 0.25150973  1.26177503]
 [ 0.33688177 -0.7556882 ]
 [-0.15493005 -0.25521361]
 [ 1.50833058  1.85502766]]
Before: [[ 1.87451371 -0.17870081]
 [-0.33886524 -1.7130757 ]
 [-0.3370512  -0.46258247]
 [ 2.66102891  2.86960713]]
After: [[ 1.12332272  0.47974993]
 [ 0.02138428 -1.08331436]
 [-0.00622494 -0.20988355]
 [ 2.64596204  3.40642086]]
Before: [[ 1.28433413  0.32900924]
 [ 1.15661818 -1.08530179]
 [-0.3009994  -1.11427677]
 [ 3.50842314 -0.84545172]]
After: [[ 0.48311768  1.20889115]
 [ 0.70983933 -1.13019547]
 [-0.05591533 -0.78196075]
 [ 1.29775564  1.44957576]]
Before: [[ 5.36994983 -1.30891132]
 [ 0.06387258 -0.12266318]
 [-1.75492103 -0.90012806]
 [ 1.21592284 -1.20736384]]
After: [[ 1.52221433 -0.51884549]
 [ 0.24517307  0.18027119]
 [-0.07054139 -0.03397671]
 [ 0.19396943  0.3211259 ]]
 Error rms: 14.259874043784437


extrapolation
Before: [[ 1.10549534  0.94205706]
 [ 0.38459908  0.10009665]
 [-0.52916509  0.65916718]
 [ 0.41910992  1.42923495]]
After: [[ 0.26410625  0.91535381]
 [-0.2170135  -0.12856285]
 [-0.17361796  0.70836189]
 [ 0.68263786  1.3558424 ]]
Before: [[ 1.26577207 -0.24933065]
 [-0.36102809 -0.03748033]
 [-0.88736632 -0.21815889]
 [-0.05670404  1.89570036]]
After: [[ 1.20806     0.29509809]
 [-0.09335464  0.33622245]
 [-0.68996668 -0.03622882]
 [ 0.08168241  1.72633569]]
Before: [[ 1.87267026 -0.43424609]
 [ 0.8304084  -0.61009215]
 [-0.06024319 -0.51719503]
 [ 0.54228065  0.44384514]]
After: [[ 1.73376112 -0.27445751]
 [ 0.18963113 -0.38012158]
 [ 0.20981119 -0.37747201]
 [ 0.67857022  0.45354735]]
Before: [[ 0.31690729 -0.74921777]
 [ 0.02341229  1.05354523]
 [-0.53735322  0.84936972]
 [-0.49602071 -0.25204321]]
After: [[-0.32714172 -0.48976595]
 [-0.2699884   0.62519838]
 [-0.11572414  0.85491961]
 [-0.16582873  0.04592923]]
 Error rms: 7.89327055490626
